# Modules, pragmas and cabal

## Outline

* Modules
  - Prelude module

  - Working with base modules 

  - Creating our own modules

  - Compiling Haskell programs

* Language pragmas

* Using Cabal
  - Installation of packages with cabal
  
  - Building a project with cabal

In this lesson, we will learn how you can work with Haskell modules, pragmas and Cabal. These are the tools necessary to build a full-fledged Haskell application so that we can finally build a complete project. We will see that modules will allow us to manage code better and let us reuse code that others have built! Pragmas will let us tweak the behavior of Haskell and how it should work. Lastly, Cabal will help us manage a complete project and all its possible dependencies!

## Modules

**What are they?**

In any programming language, the code that we write can get quite complex. In these case, we need ways to manage our code, in Haskell we do this with modules.

At its core, a Haskell module is just a collection of functions, types and type classes defined in a file that can be imported and used in other Haskell files.

As an example, in the design of a card game program, it is good practice to strictly separate the code that defines the game logic and the code that renders the game. In Haskell, this is achieved with making a module for each of these two functionalities.

The logic module of this game might contain types of the cards and function that define a valid card action. This, while the module that renders the game contains functions and types that capture the state and the transition of a game.

Here the two modules working together create the application, and by separating the code we also gain improved readability and the prevention of spaghetti code!

This separation of the two also allows code to be reused. If we later want to make another dice game, we could reuse the module that renders the game, saving use a precious time.

<div class="alert alert-block alert-info">
    Notice that the reuse and compossibility of modules is aligned with the virtues of the functional programming paradigm! Similar to functions, modules are also piecewise defined. Moreover, as with function, modules allow us to hide complexity and only expose the user of the module with the necessary types and functions. This means that helper functions that are necessary for the functionality of the module, but not the usage, can be hidden!
</div>

**How to use them?**

Actually, you already know how to use modules! In some of the previous lessons, we secretly have been using them already.

For example, in the previous lesson, we import the function `listDirectory` from the module `System.Directory` via the line
```haskell
import System.Directory (listDirectory)
```

Here, the dot separates the module `Directory` as a submodule from the parent module `System`. The `System` module exposes more submodules, for example `System.FilePath`. This naming convention with dots organizes and categorizes modules so that they are easier to find.

If we deconstruct this line, we see that it consists of 3 parts. 

- The first is the syntax `import` that declares that an import will follow.
- Next is the specific module by its name `System.Directory`.
- And lastly, we specify that we only want to import the function `listDirectory` from this module in brackets.

But this is not everything you can do. Instead of only importing that function, we could have import every function and type it exposes by omitting the brackets and its content completely

Besides these three markers for import, there are two more. The first extra thing we can add to an `import` is qualifying the import. This means that the usage of that import and its imported function and types can exclusively be used by specifically mentioning the import at usage. This is especially handy if modules export function that have the same name! Then the compiler may need some help to decide which function from which module it needs. For example,

```haskell
import qualified System.Directory (listDirectory)

getFiles :: IO [FilePath]
getFiles = System.Directory.listDirectory "." 
```

Now, some modules may have a long name or are embedded as multiple submodules (for example `System.Posix.Directory`). To make things easier while coding, we can import these modules and rename them. For example,

```haskell
import qualified System.Directory (listDirectory) as SysDir

getFiles :: IO [FilePath]
getFiles = SysDir.listDirectory "." 
```

<div class="alert alert-block alert-warning">
    Notice that module names are capitalized, if you are renaming them, this new name has to be capitalized as well!
</div>

To reiterate, modules are just Haskell files. To indicate this, a `.hs` file has a module definition statement at the beginning of the file, followed by some Haskell code. For example, a Haskell file might start with the line

```haskell
module Tree where

-- the rest of the haskell code here
```

If we dissect this line of code, it consists again of three things

- The first is the syntax `module` that declares that this is a module.
- Next is the specific name that this module will use.
- And lastly, we end the module declaration with a `where` that indicates that the rest that follows is code of the module

It is also good practice to name the Haskell file after the name of the module. 

This module can now be imported into other Haskell files. If done so, all the code from the module `Tree` becomes available to the other Haskell file. 

This is because currently the above module declaration does not specify its exports, so by default it exposes all its code. 

In case you want to restrict access to some functionality, you can specify what is exported by making it explicit as

```haskell
module Tree (function1, function2, myDataType) where
```

<div class="alert alert-block alert-info">
    To discover what a module exposes, you can either look at the source code or test it out in GHCi. In GHCi import the module with a new name, for example <code>import System.Directory as SD</code>. To explore what `SD` exposed, you can now type <code>SD.</code> inline and press tab to autocomplete to discover all possibilities. In lesson 15 (learning on your own) we will explain how you can look up these things in a streamlined way.
</div>

### The Prelude module

When working in GHCi or writing your own Haskell code, some functions are available by default, for instance `head`, `sum` and `length`.

This means you had to do nothing to have these imported or installed for them to be used.

This is because those functions are part of the standard Haskell module called **Prelude** that is imported by default. 

The word prelude means an introduction to something more important, which is the code and modules you will write.

You can find a list of all the functions contained in **Prelude** on **Hackage** [(1)](https://hackage.haskell.org/package/base-4.17.0.0/docs/Prelude.html). We will explain in lesson 15 what Hackage is and how you can use it in general.

On the Hackage link of the prelude module provided above, you will also find the type signatures of all these functions and more.

<div class="alert alert-block alert-info">
    To view all the imported modules in a current GHCi session, use <code>:show imports</code>. GHCi will also list the current imported modules in the promt, for example the basic <code>Prelude></code> promt. If this promt gets too cluttered due to lots of import, you can change it with <code>:set promt NEW_PROMT_HERE</code>.
</div>

### Working with Base modules

Some functions in Haskell need to be imported via modules that are already locally available, besides the standard **Prelude** module. 

We call the combination of all these already locally available modules the **Base** modules (of which the prelude is one). Haskell contains many convenient modules for you to try out!

For an overview of what the Base consists of, take a look at the following foldable list [(2)](https://hackage.haskell.org/package/base-4.17.0.0/docs/index.html).

As this is an introduction to Haskell, the most important modules in this list right now are 

* Data
* Numeric
* Prelude
* System
* Text

Let's say you want to run a Haskell program that executed something from the command line by giving it some input parameters and then read them from the program. 

This is a common pattern, and it is available in the Base module `System`.

More precise, you can do this with the function `getArgs` that is part of the `System.Environment` module. For more info on this module, you can take a look at its Hackage page [(2)](https://hackage.haskell.org/package/base-4.17.0.0/docs/System-Environment.html).

```haskell
import System.Environment 

main :: IO ()
main = do
  args <- getArgs
  print $ head args
```

<div class="alert alert-block alert-warning">
The above code will not properly work in this Jupyter Notebook. This is because the <code>getArgs</code> function looks for arguments from a command line! At the end of the lesson, you will be able to execute this code locally with cabal or GHC.  
</div>

When we import a module as above, all of its types and functions that it exposes get imported. We can also specify precisely which functions and types we want to import.

For the example above, we could have imported only the function `getArgs` with the following statement instead of the whole module
```haskell
import System.Environment (getArgs)
```
If there are more functions and types we wish to import, we separate them with a comma. 

Now you might be asking yourself how to know which module to import and which function to use? The best answer is always googling what you want to achieve. 

For our example of the `getArgs` function, you could for instance write into google: `haskell get command line arguments`.

Once you find a module or function that fits your needs, you can further use Google to find a good explanation and example. 

You can also use the Haskell web services **Hackage** and **Hoogle** to learn more about them. Again, we will discuss these web services in detail in lesson 15.

### Base modules Data.Char, Data.List and Data.Bool

As an example, we will go through three often used modules that are also locally available for you to use in Haskell. We will briefly discuss what these might be used for to provide some insight in their structure.

The **Data.Char** module defines functions that deal with characters. Some of the often used functions are:

- `isDigit` (checks if a character is a digit)

- `isPunctuation` (checks if a character is a punctuation mark)

- `toUpper` and `toLower` (converts a lower case character to upper or vice versa)

- `ord` and `chr` (convert a character to ASCII code number and vice versa)

For some example code, we could have

In [ ]:
import Data.Char

isDigit '1'
isPunctuation '.'
toUpper 'a'
ord 'a'

This module is great for parsing characters and changing them.

The **Data.List** module defines functions that deal with lists. Some of the functions we have not covered yet are:

- `sort` (sorts a list from lower to upper values, if the variables can be compared to each other)

- `splitAt` (splits a list into a tuple of two lists, where the user defines the length of the first list)

- `scanl` (is similar to foldl, but it shows the value of the accumulator on each iteration instead of the result)

- `scanr` (is the right-to-left dual of scanl. The order of parameters on the accumulating function are reversed compared to scanl.)

For some example code, we could have

In [ ]:
import Data.List (sort)

myList = [4,2,1,5,3]

sort myList
splitAt 3 myList

scanl (-) 5 [1..4]
scanr (-) 5 [1..4]
foldr (-) 5 [1..4]

<div class="alert alert-block alert-info">
    The scan functions are similar to the fold functions. The difference is that the fold functions are focused on the result of the fold (a single return value). While the scan functions tell you how that result was achieved by showing its intermediate values (a list of values). So, the last element of a <code>scanl</code> list is the result of a <code>foldl</code>, and the first element of <code>scanr</code> is the same as <code>foldr</code> over the same arguments.
</div>

Besides functions, the `Data` module in Base also contains the definition of the Booleans in Haskell in `Data.Bool` [(3)](https://hackage.haskell.org/package/base-4.17.0.0/docs/Data-Bool.html). Here, a `Bool` is simply defined as a data type of either True or False
```haskell
data Bool = True | False
```

Here we can also see which basic operators are available to use on the `Bool` type, for example the `(&&)` and `(||)` functions. But also note that here the `otherwise` that we use in guards is defined as simply `True`! Remember that when using guards you are testing against conditions that are reduced as bool values where the `otherwise` statement cached all the other cases! For example,

In [ ]:
myGuardFunct :: Integer -> Integer
myGuardFunct x
            | x < 0     = -x
            | otherwise = x
myGuardFunct (-10)

### Creating your own module

Since modules are just plain Haskell files that can be imported in other Haskell files, it is easy to create a module on your own. 

In the following section, we will show you how to set up a module.

Let's say we want another version of the Prelude function `sum` that returns an error for the input of the empty list, instead of the value 0 that the Prelude `sum` returns.

To create a module that exposes such a module, we first create a Haskell file that we call `Sum.hs`. At the top of this file, we write a module statement like
```haskell
module Sum where
```

With this statement, we defined the name of our module as `Sum`, which again should start with an upper case letter.

It is good practice that the name of the module is the same as the name of the file, though this is not mandatory.

<div class="alert alert-block alert-warning">
    In the case of submodules like <code>import System.Directory as SD</code>, the module declaration is the same as the import (<code> module System.Directory where</code>), while the name of the file does not contain the string <code>"System</code>. It is common that the directory that contains the submodule <code>.hs</code> file is names after the parent module (<code>System</code>).
</div>

Then we define our own parameterized type `MyData` that can be used for displaying an error message. 

We will learn more about proper error handling in lesson 14. After that, we define our `sum` function.

```haskell
module Sum where

import qualified Prelude

data MyData a b = Error a | Result b deriving Prelude.Show

sum :: Prelude.Num a => [a] -> MyData Prelude.String a
sum [] = Error "List is empty"
sum xs = Result (Prelude.sum xs)

```

Notice that in the definition of our `sum` function, we use the prelude version of `sum` that we access by `Prelude.sum` to prevent name collision of our sum.

Now, if we were in another Haskell file and wanted to import our `Sum` module, we should use qualified import to avoid name collision with the Prelude.

```haskell
import qualified Sum (sum)
import qualified Prelude

Prelude.sum []       -- 0
Prelude.sum [1..3]   -- 6

Sum.sum []     -- Error "List is empty" 
Sum.sum [1..3] -- Result 6
```

If you defined your `sum` function with another name that would not match any default function name from Prelude, you could use a simple import statement as `import Sum`. 

This would work if our function name would be e.g. `sum1`. Then you could use the function names directly. 

```haskell
import Sum

sum []      -- 0
sum [1..3]  -- 6

sum1 []     -- Error "List is empty" 
sum1 [1..3] -- Result 6
```

In contrasts to imports restriction like above where we only imported `sum` from the module `Sum`, Haskell also gives you control over exports. That is, what does a module expose to the outside world?

In the above example, our module `Sum` exports all that is declared in its file. To take control over what the `Sum` module exports, we declare the things we want to export in the module declaration.

```haskell
module Sum (sum) where

import qualified Prelude

data MyData a b = Error a | Result b deriving Prelude.Show

sum :: Prelude.Num a => [a] -> MyData Prelude.String a
sum [] = Error "List is empty"
sum xs = Result (Prelude.sum xs)
```

This means that when importing the module `Sum`, we no longer have access to the data type `MyData` and its constructors `Error` and `Result`! Things like
```haskell
import Sum 

willNotWork :: MyData String Integer
willNotWork = Result 10
```
will fail with an error `Not in scope: type constructor or class ‘MyData’`.

If we did want to export this data type explicitly, we should have used
```haskell
module Sum (
  sum
, MyData(Error, Result) 
  ) where
```

<div class="alert alert-block alert-info">
   It is common practice to separate multiple exports by adding them to a new line.
</div>

Notice that we also added some brackets to the export of the data type `MyData` to also export its type constructors! If you do not want to make this explicit, but want to export all its constructors, you can also use `(..)` to export 

### Using modules to your advantage

We mentioned that modules gives you control over what is exposed in the module and that it can hide complexity. In this example, we will use this trick to enforce some extra constraints on a type.

To achieve this we will use *smart constructors*, this is a common Haskell pattern in a module that hides the constructors of a data type and instead expose a function with similar but constrained functionality!

In this example we want to implement a natural number type, these are integers larger than zero. A naive way would be to implement this as
```haskell
module Naturals where

newtype Nat = Nat Integer deriving Show
```

This indeed creates the namespace and a type for a `Nat` but it does not ensure that the integer is bigger than zero! 

Remember that the `Nat` after the `=` sign acts as the constructor of this type. Each object that we create of type `Nat` is initiated with this constructor. For example,

In [ ]:
newtype Nat = Nat Integer deriving Show

aNatural1 = Nat 10
aNatural2 = Nat (-10)

print $ show aNatural1
print $ show aNatural2

When deciding what to export in your module, we can choose to hide this `Nat` constructor while still exposing the type `Nat`.

But if we do this, another module that uses our `Naturals` module can never construct a `Nat` type since the constructor is hidden, so we must give an alternative.

This is where the *smart* constructor comes in. This is a function that has the same name as the constructor, but without a capital at the beginning.

In addition, this function creates a `Nat` with its proper constraint of being strictly bigger than zero!

```haskell
module Naturals (
  Nat
, nat
) where

newtype Nat = Nat Integer deriving Show

nat :: Integer -> Nat
nat n = Nat (abs n)
```

We see here that instead of exposing the full type via `Nat (Nat)` in the exports, we have access to the creation of such a type via the function `nat`.

<div class="alert alert-block alert-info">
    Notice that the type of the constructor <code>Nat</code> given by <code>Nat :: Integer -> Nat</code> is the same as the function <code>nat</code>!
</div>

In [ ]:
module Naturals (
  Nat
, nat
) where

newtype Nat = Nat Integer deriving Show

nat :: Integer -> Nat
nat n = Nat (abs n)

In [ ]:
import Naturals

correctNatural = nat (-10)
print $ show correctNatural

### Compiling Haskell programs

In this section, we will show how you can compile simple Haskell files, later we will show how you can compile a more complex project using Cabal.

When compiling a Haskell program we want it to perform some IO action to be useful, in general, an executable that is compiled is impure as it performs some useful action on the operating system!

This is why all executables that we will build in Haskell have an entry point of their logic that is marked as a function of type `IO ()`.

This function has no inputs and produces no outputs, but it does create a side effect.

You can compile any Haskell module using GHC. The compiler GHC will then convert the functions and the structure of the module (what it exposes) into at least two files.

These two files have the suffix `.hi` and `.o`. Here, the latter is just a C's object file. This contains all the function but compiled into a lower form that is machine understandable.

The former `.hi` is an interface file, it tells GHC how the `.o` file can be used if the module were to be used by another Haskell file. In this way, GHC can reuse and compile modules only when they change! This save precious compiling time!

To compile a basic Haskell file like our above `Sum.hs` module, we use in a shell
```bash
ghc Sum.hs
```
with results in the artifacts
```bash
ls
Sum.hi  Sum.hs  Sum.o
```

As we mentioned, most useful programs have side effects and produce an executable that we call its binary, the machine code that executes the actions.

You can compile such a Haskell program, that is a Haskell file with your main logic, also using GHC. This file can import other Haskell files that define modules as well (like our `Sum` module).

To compile an executable with GHC, your main file must have the name `Main.hs` and it must contain a `main :: IO ()` function which is the entry point for the compiler. 

When the executable is run, this is the function that will be evaluated at runtime. For example, consider the file `Main.hs`
```haskell
module Main where

import qualified

main :: IO ()
main = do print . show $ Sum.sum []
```

You compile your program from the command line like this:
```
ghc Main.hs
```

This will create three new files instead of two!
```bash
ls 
Main  Main.hi  Main.hs  Main.o  Sum.hi  Sum.hs  Sum.o
```

<div class="alert alert-block alert-warning">
    Notice that the output of this command does not recompile the <code>Sum.hs</code> module, rather it uses the already existing <code>Sum.hi</code> and <code>Sum.o</code> files. Try removing all the build artifacts like the <code>Main</code> and the <code>.o/.hi</code> files.
</div>

Here the first is the executable that we can run, on a Unix system, this can be done via the command `./Main`. 

The rest of the files are similar to the compilation of any other module.

The code from the additional Haskell files is included automatically if the files are sitting in the same directory as the Main.hs file.

Also, all the files have to have module declaration statements and the Main.hs file has to import them for them to be included in the compile process.

<div class="alert alert-block alert-info">
    GHCi the Haskell REPL allows you to load a Haskell file with the <code>:l</code> command. There, it is not relevant whether the file has a main function or not. Once the file is loaded into GHCi you can call any of the functions or types defined in the file and test them if they work as you expected. If you load a <code>main.hs</code> file into GHCi that imports some user-defined modules, they will also be included as in the compilation process.
</div>

## Language pragmas

Language pragmas or also called language extensions, are a way to modify how the Haskell compiler interprets your code. 

Because of this, you can say that they add or alter some functionality of your Haskell code.

The syntax to add a pragma to a Haskell file is by adding the statement below at the top of the file:
```
{-# LANGUAGE pragme_name #-}
```
Pragmas are not like modules because they do not bring any new functions to Haskell, but just code functionality.

Let's look at an example where we use the **NoImplicitPrelude** pragma. 

When this pragma is added to your file, it prevents the default import of the **Prelude** module.

Because of this, if you can define your own versions of Prelude functions without having a name collision.

One reason why this would be useful is you do not want any partial functions.

In [ ]:
{-# LANGUAGE NoImplicitPrelude #-}

import GHC.Base (String)
import GHC.Show (Show)

data Data a b = Error a | Result b deriving Show

head :: [a] -> Data String a
head [] = Error "Empty list!"
head (x:xs) = Result x
 
tail :: [a] -> Data String [a]
tail [] = Error "Empty list!"
tail (x:xs) = Result xs

In the example above we see because we prevent the import of Prelude we have to manually import the `String` type and the `Show` type class. 

Another reason is that you want to avoid using some of Prelude's lazy functions. So, you define them such that they are strict.

To achieve this, you can use the **BangPatterns** pragma that lets you add an exclamation mark in front of variables to evaluate them strictly.

In [ ]:
{-# LANGUAGE BangPatterns #-}
{-# LANGUAGE NoImplicitPrelude #-}

foldl :: (b -> a -> b) -> b -> [a] -> b
foldl f = go 
  where
    go !z []     = z
    go !z (x:xs) = go (f z x) xs

Another language pragme we can look at is the **DuplicateRecordFields** pragma.

It allows you to define various record syntax types that contain duplicated function names.

In [ ]:
{-# LANGUAGE DuplicateRecordFields #-}

data UserAge = UserAge { name :: String
                       , age :: Int }

data UserHeight = UserHeight { name :: String
                             , height :: Int }                     

The last example we will show is how to use the **TypeApplications** pragma which allows you to use visible type application in expressions.

It's just another way of how to define a type of variable. You use it such that you add the `@` symbol with the type name before the variable. 

In [ ]:
{-# LANGUAGE TypeApplications #-}

myInt = read @Int "1"
myFloat = read @Float "1.23"

You can use this notation also when calling functions. In the example below we define the `Number` type that holds either a whole number, or a decimal number or a string.

The parse function then takes in a string and analyzes it if it represents a whole number, a decimal number or if it's not a number at all.

In [ ]:
{-# LANGUAGE TypeApplications #-}

data Number a b = WHOLE a | DECIMAL b | NAN String deriving Show

parse :: (Read a, Read b) => String -> Number a b
parse inp = if isNumber
            then if isDecimal
                 then DECIMAL $ read inp 
                 else WHOLE $ read inp
            else NAN "Not a valid number"
    where validChar = ".0123456789"
          isNumber = all (&& True) $ map (`elem` validChar) inp
          isDecimal = '.' `elem` inp

longWhole = "9223372036854775808"
longDecimal = "1.23456789"

parse @Int @Float longWhole
parse @Integer @Float longWhole
parse @Int @Float longDecimal
parse @Int @Double longDecimal

When we call the `parse` function, we specify the types for the variable `a` and `b`. 

This enables us to choose what kind of precision we want when parsing a whole or a decimal number.

As a side note, we state here that the `@` symbol can also be used for referencing lists, which has nothing to do with type applications.

In the example below, the `counts` function takes in a list of numbers and returns a list of tuples with the unique number and their appearance counts.

When we pattern match the list with `(x:xs)` we also say `list@` that creates a variable named `list` which is equal to the entire list `x:xs`.

In [ ]:
newtype Count = Count Int deriving Show

counts :: Eq a => [a] -> [(a, Count)]
counts [] = []
counts list@(x:xs) = (x, Count $ length (filter (== x) list)) : counts (filter (/= x) list)

counts [1,2,1,4,5,2,4,3]

There are also many other pragmas that you can add to your code. Some of them are:

- **OverloadedStrings**: Lets you define variables of data types that store text by directly using a string instead of applying a helper function to it.

- **TemplateHaskell**: Provides tools for Haskell meta-programming, which means that the code generates other code. It is also used in Plutus.

- **ViewPatterns**: Allows for more sophisticated pattern matching.

A list of all language extensions can be found on Hackage [(2)](https://hackage.haskell.org/package/template-haskell-2.19.0.0/docs/Language-Haskell-TH.html#g:5).

## Using cabal

### Introduction

**What is cabal?**

It is a Haskell package management tool that can be used from the command line, similar as **pip** for Python or **npm** for JavaScript. 

The name **cabal** stands for *Common Architecture for Building Applications and Libraries*.

**What is a Haskell package?**

A Haskell **package** is a collection of Haskell files that defines modules. The modules contain code that offer related functionality. 

The reason why there are multiple modules contained in a package are: 
- a developer might not need all the functionality from the package and wants to import only certain modules

- it is easier to maintain the package if the code is contained in multiple modules instead of one 

**Why do we need cabal?**

Imagine we have a file that contains the molar concentrations of vitamin C, vitamin E and copper for a person's blood. 

The units are `mol/L` and the numbers have scientific notation.
```
Vitamin C: 40e-6
Vitamin E: 50e-6
Copper:    15e-6
```

Now, when you read the file and extract the strings that contain the numbers, you cannot parse them with the `read` function.

After some search on Google, you find out there is a module called **Data.Scientific**, that allows you to parse the numbers in scientific format.

The parsing can be done with the `read` function:
```haskell
read "40e-6" :: Scientific
```

But when you try to import this module, you realize that you get an error.

This is because the package **scientific** which contains this module is not installed by default when you install **GHC**.

Only **packages** that contain commonly used modules come with the default installation of Haskell. Some of them are **base**, **text** and **time**.

For other packages that you might need, **cabal** can help you to install them on your OS.

### Managing packages with cabal

**How to check if a module / package is available by default?**

To check whether a module can be imported in Haskell, simply start GHCi and type `import module_name`. 

If the package containing the module is not installed, you will get an error. 

You can also use the TAB button for auto-completion. For instance, type `import Data.A` and hit TAB. You will get a list of all modules that start with `Data.A`. 

For packages, you can display the list of installed packages on your OS with the command:
```
cabal list --installed
```

**How to install a Haskell package?**

When **cabal** installs a package, it downloads it from [Hackage](https://hackage.haskell.org/). As said before, we will talk about Hackage in lesson 15.

In case you already know which package you need to use, you can install it with one of the commands below.

To install a package on your OS, you can use the command:
```
cabal install package_name
```

It could happen that cabal will complain if the package is in form of a library. In this case, use the command:
```
cabal install --lib package_name
```

If you download a tar.gz file of a package, you can install the package locally with the command:
```
cabal install ./package_name.tar.gz
```

An example of how to install the **scientific** package is:
```
cabal install --lib scientific
```

**How to remove a Haskell package?**

Currently, **cabal** does not know how to uninstall packages.

You can install the **cabal-uninstall** package that lets you remove Haskell packages.
```
cabal-uninstall package_name
```

### Building a project with cabal

We learned now how to use **cabal** to manage Haskell packages. But **cabal** can also be used for another purpose.

You can use it to build Haskell projects instead of compiling the source code with the **ghc** command.

Cabal uses a `.cabal` file that describes the building process of a project.

The reasons for building projects with **cabal** instead of **ghc** are:
- Cabal can handle a project structure that contains files in multiple folders.

- In the cabal file, you can specify for each folder separately: 
  - Which of the installed packages you want to import.

  - Which packages cabal should add as build dependencies.<br>
    They will be used in the compiling process, but not installed on the OS.

  - What the folder contains: the main executable file, library files or test files.

- The build is more clean. All files created from the build are put in the `dist-newstyle` folder.

- You can create a package with cabal and distribute it to other developers.

**Creating new project**

To create a new project in cabal, create an empty folder, move into it and use one of the following commands:

Creates a simple project:
```
cabal init
```

Creates a project by asking you multiple questions where you can choose from a set of parameters:
```
cabal init --interactive
```
**NOTE**: Initially, you should say "no" for a simple project. Otherwise, the command runs as in the first case.

The commands above will create a cabal file and some folders with files depending on the options you specified.

**Explaining the cabal file**

Let's have a look at the contents of an example cabal file: 
```
cabal-version:      2.4
name:               test
version:            0.1.0.0

-- A short (one-line) description of the package.
-- synopsis:

-- A longer description of the package.
-- description:

-- A URL where users can report bugs.
-- bug-reports:

-- The license under which the package is released.
-- license:
author:             Luka Kurnjek
maintainer:         luka.kurnjek@iohk.io

-- A copyright notice.
-- copyright:
-- category:
extra-source-files: CHANGELOG.md

executable test
    main-is:          Main.hs

    -- Modules included in this executable, other than Main.
    -- other-modules:

    -- LANGUAGE extensions used by modules in this package.
    -- other-extensions:
    build-depends:    base 
    hs-source-dirs:   app
    default-language: Haskell2010
```

The fields in the beginning are self-explanatory. You can remove any comments that start with `--`. 

The name of your project is set to the name of the folder on which you created the project. 

This name will also be used for the name of the package if you want to build and distribute it. 

In case a package with the name you set already exists on Hackage, you will be notified. 

After the initial 5 fields, the `extra-source-files` field specifies the file that contains the copyright notice.

Then the fields that specify the project folder types are defined, which also contain build instructions.

One of the more often used fields are:
- `executable`: defines information for the executable part of project

- `library`: defines information for the supporting libraries of the project

- `test-suite`: defines information for the testing part of the project 

For the `executable` filed, you first need to add the name of the project and then configuration options:
- `main-is` defines the Haskell file which gets executed when the program is started (usually Main.hs). 

- `build-depends` defines the packages needed to build this project (base package is included by default)

- `hs-source-dirs` defines the name of the folder where the main executable file is residing

- `default-language` specifies the name of the Haskell release you want to use.

Also, other configuration options can be added to the executable section as:
- `import` defines which of the already installed modules to import when compiling the main executable

- `ghc-options` defines which compiler flags to use when building this code

Cabal supports equality and inequality operators for comparing package versions. 

For instance, when you define the `build-depends` you can set conditions for library versions. 

You could for write: 
```
build-depends: base ^>=4.14.3.0
```

The carrot operator `^` is used to treat `^>= x.y.z` as identical to `>= x.y.z && < x.(y + 1)`. 

So in our case, we could write the above statement also as: 
```
base >= 4.14.3.0 && < 4.15
``` 

For the `library` and `test-suite` fields, the same options and rules apply as for the `executable` field.

Common folder naming practices for these fields are:
- `src` or `lib` for `library` field

- `test` for `test-suite` field

**Building and running your project**

All the command in this section need to be performed inside the project folder at the top level.

To build your project, you run the command:
```
cabal build
```

To run your project, you run the command:
```
cabal exec project_name
```

You can do both actions also with one command:
```
cabal run
```

## That's it for today!